In [1]:
import requests
import pandas as pd
from pymongo import MongoClient
from bs4 import BeautifulSoup
import numpy as np


In [6]:
# xml base - search festival example
url = 'http://api.visitkorea.or.kr/openapi/service/rest/KorService/searchFestival'
key = '?ServiceKey=S256yGjpf7ug3eDtNRoJBzPLPIED9Mzp3XfMp8aaoRB/FGnDuzloLHkpvLvTyddzf00SKndA/1naWcmH2ao5jg=='
option = '&contentTypeId=32&areaCode=4&sigunguCode=4&MobileOS=ETC&MobileApp=AppTest'
response = requests.get(url+key+option)

xml_obj = BeautifulSoup(response.content,'lxml-xml')
tp = xml_obj.find_all('item')
columns = [tp[0].find_all()[i].name for i in range(len(tp[0]))]

import numpy as np
pd.DataFrame(np.array(tp).squeeze(),columns=columns)

# 관광데이터 부분은 크기가 좀크고, 호출방식이 json보다는 까다로워서 데이터 필요한것에 대한 의논을 간략하게하고, 뽑아봅시다. 

,addr1,addr2,areacode,cat1,cat2,cat3,contentid,contenttypeid,createdtime,eventenddate,...,firstimage,firstimage2,mapx,mapy,mlevel,modifiedtime,readcount,sigungucode,tel,title
0,대구광역시 동구 동대구로 475,(신천동),4,A02,A0207,A02070200,2759598,15,20211020094120,20211212,...,http://tong.visitkorea.or.kr/cms/resource/96/2...,http://tong.visitkorea.or.kr/cms/resource/96/2...,128.6249099977,35.8729035334,6,20211020094120,0,4,053-740-0421,대구 건축문화기행 캠페인
1,대구광역시 동구 팔공로45길 26,(봉무동),4,A02,A0207,A02070200,2377784,15,20160412153323,20211212,...,http://tong.visitkorea.or.kr/cms/resource/11/2...,http://tong.visitkorea.or.kr/cms/resource/11/2...,128.6379131218,35.9189735573,6,20211118131701,10246,4,053-721- 7400,2021 디지털패션문화페스티벌


In [ ]:
# write
my_client = MongoClient('mongodb://localhost:27017')
mydb = my_client['yosul_lamp']
mydb['programs'].insert_many(pd.DataFrame(response.json()['data']).to_dict('records'))

# read
my_client = MongoClient('mongodb://localhost:27017')
mydb = my_client['yosul_lamp']
pd.DataFrame(mydb['programs'].find()).drop('_id',axis=1)